In [ ]:
'''
Pre-release Notice

This repository contains code associated with our ongoing research project titled "An insight into the technological landscape and competitive situation of biodegradable PBAT industry based on multidimensional patent analysis". The code is being made available for **review purposes only** and is subject to the following restrictions:

1. Non-commercial use only: This code may only be used for academic or non-commercial purposes.
2. No redistribution or modification**: Redistribution or modification of this code is not permitted until the associated research paper has been officially published.
3. Temporary access: The code in this repository is subject to updates and may change without notice until the final release.

After the publication of the corresponding research paper, we plan to release the code under a more permissive open-source license (e.g., MIT License).

For any questions or specific permissions, please contact zhangx2293@gmail.com with the subject "Pre-release Code Inquiry".

Written by Xiang Zhang
'''

In [1]:
import pandas as pd
import networkx as nx
from collections import Counter

data = pd.read_excel('data.xlsx')

In [2]:
data['IPC主分类号'] = data['IPC主分类号'].str[:4]
data['[标]原始申请(专利权)人'] = data['[标]原始申请(专利权)人'].str.split('|').str[0]
data = data.drop([0, 1])
data

,序号,公开(公告)号,标题,专利类型,法律状态/事件,申请日,授权日,发明人,简单同族专利申请数量,摘要,...,引用专利,引用专利数量,非专利引用文献数量,许可类型,5年内被引用次数,质押人,申请年,发明人(全英),摘要(全英),标题(全英)
2,1813,AT1005298T,Teilchenförmige konstrukte zur freisetzung von...,授权发明,授权 | 放弃-视为放弃,2005-07-19,2018-06-06,"MAYER, LAWRENCE, D. | PRUD'HOMME, ROBERT, K. |...",14,Particulate constructs stabilized by amphiphil...,...,-,0,0,-,0,-,2005,"MAYER, LAWRENCE, D. | PRUD'HOMME, ROBERT, K. |...",Microparticle constructs stabilized by an amph...,Technical forms of structural liberalization
3,1476,AT1070095T,Neue synthetische antimikrobielle peptide,授权发明,授权 | 放弃-视为放弃,2015-08-13,2018-11-28,"KENDURKAR, SHUCHISHWETA VINAY | SENGUPTA, DURBA",6,The present invention provides anti-microbial ...,...,-,0,0,-,0,-,2015,"KENDURKAR, SHUCHISHWETA VINAY | SENGUPTA, DURBA",The present invention provides antimicrobial c...,New synthetic antimicrobial peptides
4,1782,AT1177292T,Kohlenstoffnanoröhrchenherstellungsverfahren z...,授权发明,授权 | 放弃-视为放弃,2013-01-28,2019-09-11,"LEWIS, GARY",10,A carbon nanotube production system is used fo...,...,-,0,0,-,0,-,2013,"LEWIS, GARY",carbon nanotube production system is used to i...,Microbial stimulation and engine emission limits
5,1679,AT1212033T,Hochenergetische zusammengesetzte peptid-selen...,授权发明,授权 | 放弃-视为放弃,2013-12-26,2019-12-11,"CUI, JIANJIU | FU, HUADIAO",10,The present invention relates to a high-energy...,...,-,0,0,-,0,-,2013,"CUI, JIANJIU | FU, HUADIAO",The present invention relates to a high-energy...,"Advanced studies, research and applications of..."
6,1342,AT1301002T,Mikro- oder nanokapseln mit photokatalytischen...,授权发明,授权 | 放弃-视为放弃,2016-07-05,2020-08-12,"MACEDO TAVARES, CARLOS JOSÉ | GOUVEIA MARQUES,...",12,The present invention lies within the field of...,...,-,0,0,-,0,-,2016,"MACEDO TAVARES, CARLOS JOSÉ | GOUVEIA MARQUES,...",The invention belongs to the field of producin...,Mikroder Nanokapseln mit photocatalystchen eig...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2588,1342,ZA201502696B,Pesticide nano-suspension,授权发明,授权,2015-04-21,2016-06-29,"MISSELBROOK, JOHN | DUNN, JEFF",17,"Many active agents, such as insecticides, herb...",...,-,0,0,-,0,-,2015,"MISSELBROOK, JOHN | DUNN, JEFF","Many active agents, such as pesticides, herbic...",Pesticide Nanosuspension
2589,778,ZA201907268A,Pesticide,发明申请,公开,2019-11-01,-,"TEO, ENG SOON | MURPHY, BROOK C",17,The present invention relates to a pesticide n...,...,-,0,0,-,0,-,2019,"TEO, ENG SOON | MURPHY, BROOK C",The present invention relates to a pesticide n...,pesticide
2590,504,ZA202103919B,Nano-pcr detection kit for infectious bovine r...,授权发明,授权,2021-06-08,2021-10-27,"GUO, LI | WANG, ZHENJUN | ZHAO, LIFENG | LUO, ...",1,The present invention discloses a nano-PCR det...,...,-,0,0,-,0,-,2021,"GUO, LI | WANG, ZHENJUN | ZHAO, LIFENG | LUO, ...",The present invention discloses a nano PCR det...,Infectious bovine rhinotracheitis virus nano-P...
2591,358,ZA202203073B,Graphene-like biochar loaded with NANO zero-va...,授权发明,授权,2022-03-15,2022-05-25,FANG LIPING | LIU KAI | RAN QIWANG | MA YIBING,1,This invention provides Graphene-like biochar ...,...,-,0,0,-,0,-,2022,FANG LIPING | LIU KAI | RAN QIWANG | MA YIBING,The present invention provides a graphene-like...,Graphene-like biochar loaded with nano zero-va...


In [3]:
# edges list
edges = []
for row in data.iterrows():
    _, patent = row
    cited_patents = patent['被引用专利'].split(' | ')
    for cited_patent in cited_patents:
        if cited_patent != '-':
            edges.append((patent['公开(公告)号'], cited_patent))

# Count the occurrences of each string in the first column
string_counts = Counter(edge[0] for edge in edges)

# Keep edges where the citation count is greater than or equal to 10
filtered_edges = [edge for edge in edges if string_counts[edge[0]] >= 10]

# Create a directed graph
G = nx.DiGraph()
G.add_edges_from(filtered_edges)
G_patent = G.reverse()

# Generate a weighted adjacency matrix
adj_matrix = nx.to_pandas_adjacency(G, weight='weight')

# Save the adjacency matrix to a CSV file
adj_matrix.to_csv("cited_network.csv")

In [4]:
# centrality
betweenness_centrality = nx.betweenness_centrality(G_patent)
closeness_centrality = nx.closeness_centrality(G_patent)

# to DataFrame
bc_df = pd.DataFrame(list(betweenness_centrality.items()), columns=['Node', 'Betweenness Centrality'])
cc_df = pd.DataFrame(list(closeness_centrality.items()), columns=['Node', 'Closeness Centrality'])

# save to csv
bc_df.to_csv("betweenness_centrality_patent.csv", index=False)
cc_df.to_csv("closeness_centrality_patent.csv", index=False)